<a href="https://colab.research.google.com/github/rajprakashbale1/rajprakashbale1/blob/main/Q3%264.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('diabetes.csv')

In [ ]:
df.info() ## No null values, so we proceed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
## Correlation check
df.corr('pearson') ## No indications of huge correlations

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [ ]:
def in_out_split(df_org):
    x_cols = [x for x in df_org.columns if x!='Outcome']
    X = df_org[x_cols]
    y = df_org['Outcome']
    return X,y

X,y = in_out_split(df)

In [ ]:
def formula_manifest(X,y):
    rhs = '+'.join(X.columns)
    lhs = y.name+'~'
    return(lhs+rhs)

In [ ]:
import statsmodels.formula.api as smf
formula_ = formula_manifest(X,y)
model = smf.logit(formula=formula_,data=df).fit()
print(model.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Optimization terminated successfully.
         Current function value: 0.470993
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                Outcome   No. Observations:                  768
Model:                          Logit   Df Residuals:                      759
Method:                           MLE   Df Model:                            8
Date:                Fri, 21 Jan 2022   Pseudo R-squ.:                  0.2718
Time:                        21:15:58   Log-Likelihood:                -361.72
converged:                       True   LL-Null:                       -496.74
Covariance Type:            nonrobust   LLR p-value:                 9.652e-54
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -8.4047      0.717    -11.728      0.000      -9.809

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

## Find the threshold that gives maximum accuracy

def threshold_calc(X,y,tmodel):
    thresh = 0.3
    orgs = y
    max = 0
    thresh_goat = 0
    preds_p = tmodel.predict(X)
    while (thresh<0.7):
        preds = [1 if x > thresh else 0 for x in preds_p]
        accuracy = accuracy_score(orgs,preds)
        if accuracy > max:
            max = accuracy
            thresh_goat = thresh
        thresh = thresh + 0.001
    return max,thresh_goat
max,threshold = threshold_calc(X,y,model)

df['predictions_prob'] = model.predict(X)
df['predictions'] = df['predictions_prob'].apply(lambda x: 1 if x>threshold else 0)
print(classification_report(df['Outcome'],df['predictions'],digits=3))
print("Accuracy of the model is: ",accuracy_score(df['Outcome'],df['predictions']))

              precision    recall  f1-score   support

           0      0.803     0.888     0.843       500
           1      0.740     0.593     0.658       268

    accuracy                          0.785       768
   macro avg      0.771     0.741     0.751       768
weighted avg      0.781     0.785     0.779       768

Accuracy of the model is:  0.78515625


In [ ]:
class stepwise():
    def __init__(self,*args):
        self.keep_ = []
        self.try_ = args[0].columns.tolist()
        self.df = pd.concat([args[0],args[1]],axis=1)
        self.label='Outcome~'
        self.execute()
        print("Features to keep:",self.keep_)
        print("Features to drop:",self.try_)

    def execute(self):
        forwards_result = self.forward()
        backwards_result = self.backward()
        if (forwards_result != None or backwards_result != None):
            if self.forward() != None:
                self.keep_.append(forwards_result)
                self.try_.remove(forwards_result)
            if self.backward() != None:
                self.try_.append(backwards_result)
            self.execute()


    def forward(self,pent=0.5):
        try:
            for i in range(len(self.try_)):
                features = self.keep_ + [self.try_[i]]
                formula_ = self.label+'+'.join(features)
                model = smf.logit(formula=formula_,data=self.df).fit()
                temp_pval = model.pvalues[self.try_[i]]
                if temp_pval < pent:
                    pent = temp_pval
                    final_feature = self.try_[i]
            return final_feature
        except Exception as e:
            return None

    def backward(self,pexit=0.5):
        try:
            if len(self.keep_) > 0:
                formula_ = self.label+'+'.join(self.keep_)
                model = smf.logit(formula=formula_,data=self.df).fit()
                for feature in self.keep_:
                    if model.pvalues[feature] > pexit:
                        pexit = model.pvalues[feature]
                        final_feature = feature
                return final_feature
        except Exception as e:
            return None
 

X = df.iloc[:,:-3]
y = df.iloc[:,-3]
stepwise(X,y)

Optimization terminated successfully.
         Current function value: 0.622532
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.526510
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.644614
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.644003
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.638548
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.599423
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.632071
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.618959
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.622532
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.526510
  

In [ ]:
features_proc = '+'.join(['Glucose','BMI','Pregnancies','DiabetesPedigreeFunction','BloodPressure','Age','Insulin'])
model2 = smf.logit(formula='Outcome~'+features_proc,data=df).fit()
print(model2.summary())

Optimization terminated successfully.
         Current function value: 0.470998
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                Outcome   No. Observations:                  768
Model:                          Logit   Df Residuals:                      760
Method:                           MLE   Df Model:                            7
Date:                Fri, 21 Jan 2022   Pseudo R-squ.:                  0.2718
Time:                        21:19:21   Log-Likelihood:                -361.73
converged:                       True   LL-Null:                       -496.74
Covariance Type:            nonrobust   LLR p-value:                 1.500e-54
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -8.4051      0.717    -11.727      0.000      -9.810

In [ ]:
X = df[df.columns.difference(['SkinThickness'])]
max,threshold = threshold_calc(X,y,model2)

df['predictions_prob_2'] = model2.predict(X)
df['predictions_2'] = df['predictions_prob_2'].apply(lambda x: 1 if x>threshold else 0)
print(classification_report(df['Outcome'],df['predictions_2'],digits=3))
print("Accuracy of the new model after stepwise regx is: ",accuracy_score(df['Outcome'],df['predictions_2']))

              precision    recall  f1-score   support

           0      0.804     0.888     0.844       500
           1      0.741     0.597     0.661       268

    accuracy                          0.786       768
   macro avg      0.773     0.743     0.753       768
weighted avg      0.782     0.786     0.780       768

Accuracy of the new model after stepwise regx is:  0.7864583333333334


In [ ]:
print("Improvement is:", (0.7864583333333334-0.78515625)*1000,"%")

Improvement is: 1.3020833333333703 %


In [ ]:
print(model.aic)
print(model2.aic)

783.2394853724977
776.6037474408696


In [ ]:
print("Model 2 or the stepwise regressed model has better AIC")

Model 2 or the stepwise regressed model has better AIC


In [ ]:
print(model.bic)
print(model2.bic)

783.2394853724977
776.6037474408696


In [ ]:
print("Model 2 or the stepwise regressed model has better BIC")

Model 2 or the stepwise regressed model has better BIC


In [ ]:
#########################################################################################
#######################Rough Work - Ignore cells from here ################################
#########################################################################################
forward([],['Age', 'BMI', 'BloodPressure', 'DiabetesPedigreeFunction', 'Glucose',
       'Insulin', 'Pregnancies', 'SkinThickness'])

# smf.logit(formula='Outcome~Age',data=df).fit().pvalues['Age']
# smf.logit(formula='Outcome~BMI',data=df).fit().pvalues['BMI']
# smf.logit(formula='Outcome~BloodPressure',data=df).fit().pvalues['BloodPressure']
# smf.logit(formula='Outcome~Insulin',data=df).fit().pvalues['Insulin']
# smf.logit(formula='Outcome~Glucose',data=df).fit().pvalues['Glucose']
# smf.logit(formula='Outcome~Pregnancies',data=df).fit().pvalues['Pregnancies']
# smf.logit(formula='Outcome~SkinThickness',data=df).fit().pvalues['SkinThickness']

Optimization terminated successfully.
         Current function value: 0.644003
         Iterations 5


0.038815803290342864

In [ ]:
## Multi Collinearity check
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df_vif):
    vif_data = pd.DataFrame()
    vif_data['feature']=df_vif.columns
    vif_data['VIF'] = [variance_inflation_factor(df_vif.values,i) for i in range(len(df_vif.columns))]
    return vif_data

df_vif = df.iloc[:,:-1]
vif_data = calculate_vif(df_vif)
print(vif_data)

## While correlation matrix doesn't indicate any correlation between any two features,
## there seem to exist a huge multi-collinearity with features -> Glucose, BP, BMI and Age

## We try binning these features to see if we can eliminate multicollinearity here

df_org = df.copy() # Savig a copy just in case

def add_categories(df,colName):
    catColName = colName + '_cat'
    df[catColName] = pd.qcut(df[colName],q=4,labels=[1,2,3,4])
    df.drop([colName],inplace=True,axis=1)
    return df
# df['Glucose_cat'] = pd.qcut(df['Glucose'],q=4,labels=[1,2,3,4])
# df['BP_cat'] = pd.qcut(df['BloodPressure'],q=4,labels=[1,2,3,4])
# df['BMI_cat'] = pd.qcut(df['BMI'],q=4,labels=[1,2,3,4])
# df['Age_cat'] = pd.qcut(df['Age'],q=4,labels=[1,2,3,4])
# df.drop(['Glucose','BloodPressure','BMI','Age'],axis=1,inplace=True)

df_cat = add_categories(df,'Glucose')
df_cat = add_categories(df,'BloodPressure')
df_cat = add_categories(df,'BMI')
df_cat = add_categories(df,'Age')
df_cat